# Exploratory Analysis: $\Phi$* Structures

Here, we have two data set files (one for each channel): ERP values from channel 294, and from channel 187. I am unaware of the context of this data (and I assume that the context is not entirely necessary); the important point is that the data is stored in a variable named "allERP", which consists of a 1x5 array; each element of this array itself contains ERP values in a roughly 115x3054 (trial? x sample) array. The sampling rate was 2034.5 Hz from what I can gather from the DEMO_ERP_new.m script. 

Our goal is to calculate trial-by-trial $\Phi$\* structures for the two-channel subsystem 187-294 and visually present mean (across all trials) $\Phi$\* values over time. The background of $\Phi$\* and integrated information theory (IIT) is outside the scope of this notebook though suffice to say that they are a theoretical measure of consciousness by quantifying the information a system possesses "beyond the information of its constituent parts".

*Question for Nao* Are channels 187 and 294 unipolar or bipolar channels? Should I do re-referencing?

I have originally written this exploratory analysis in Julia. Julia is a relatively new language that purportedly mixes the ease of Pythonic-like syntax (though the syntax is actually much more similar to Matlab/Octave) with the speed of C. There are many more cool things about Julia that other people will be able to tell you much more about, but the importance to data science is as promising open source language once its userbase grows. The reason I did this was simply because I had Julia installed on my laptop and not Matlab/Octave and I did not want to install another language when Julia should suffice for now - and, of course, it's nice to try new things).

If you'd like to see just how similar Julia syntax is to Matlab, check out http://cheatsheets.quantecon.org/. 

Once finished, I will rewrite this analysis in Matlab. Since the syntax is so similar, this should not be too arduous.

# Data Loading

First, we simply load the data. We make use of a package - "MAT" (Kornblith & Holy) - to read the .mat file the data is stored in. If you do not have this package, then you will have to run `Pkg.add("MAT")` in Julia before proceeding.

In [1]:
using MAT

We will want to use both channel 187 and channel 294 for our two-channel subsystem so, of course, we load both sets of data.

Just a quick note here - Julia scripting doesn't require the use of semicolons to suppress console output, but writing to the interactive shell *does*. Since we are using a Jupyter notebook here, I will (sometimes when I want to suppress output) use semicolons but they're not crucial.

In [2]:
file_187 = matread("153_4sessions_li187.mat");
file_294 = matread("153_4sessions_li294.mat");

These files are read into Julia as a dictionary - you can think of this to be roughly analagous to a structure 1x1 structure with possibly many fields (depending on how big the dictionary is). Each Matlab structure field is like a Julia dictionary "key" - you can access a dictionary by indexing using a key (which is often a string) just like you can index a structure using a field.

Let's have a look what keys are in our file_187 dictionary.

In [3]:
println(keys(file_187))
println(keys(file_294))

String["allERP"]
String["allERP"]


Both data files have a single key - "allERP" (which Julia is telling us is of type "String"). Let's see what's contained in each.

In [4]:
data_187 = file_187["allERP"]
data_294 = file_294["allERP"]

println(typeof(data_187))
println(size(data_187))

Array{Any,2}
(1,5)


Okay, as expected, our variables data_187 and data_294 are storing a 1x5 array. Let's have a look at what's stored in each in each of the five elements.

In [5]:
for i in 1:5
    println(typeof(data_187[i]))
    println(size(data_187[i]))
end

Array{Float32,2}
(118,3054)
Array{Float32,2}
(113,3054)
Array{Float32,2}
(116,3054)
Array{Float32,2}
(115,3054)
Array{Float32,2}
(111,3054)


Again, as expected, each of the five elements is itself an array which seems to contain our actual data in a (roughly) 115x3054 

From what I can gather from the tutorial materials, each of the 5 elements corresponds to a "Class" of visual stimuli (upright faces, inverted faces, houses, tools, Mondrian patterns) and each row of the subsequent arrays corresponds to the actual trials.

So now, to clarify, our loaded data is structured as follows:

* data_187 and data_294 are 1x5 arrays (1 x visual stimulus class)
     * Each array contains a roughly 115x3054 array (trial x sample) of raw ERP values

# Calculating Covariance Matrices on a Single Trial

To calculate $\Phi$\*, we first need to bin the data and compute covariance matrices in each time bin. Let's consider data from only the first class at this stage.

We have two 118x3054 matrices of ERP values. We will bin the data into bin widths $T$ for each trial to estimate the covariance matrices using the following formula:

$Cov(X, Y) = \frac{(X - E(X)) \times (Y - E(Y))}{N}$

for channel x sample matrices X and Y, where E(X) and E(Y) denote the mean values across the 2nd dimension of that matrix. In short, the mean of all the products of X deviances and Y deviances from their respective mean values.

As an example, let's consider the very first time bin, say, from sample 1 to 200, for the very first trial.

In [6]:
# Specify T and tau values
T = 200;
tau = 8;

In [7]:
current_bin_187 = data_187[1][1, 1:T];
current_bin_294 = data_294[1][1, 1:T];

Now, for convenience, we will define the past and present states for a time bin to be wholly contained *within* that time bin - i.e. for a time bin width 200, the past and present states will both contain 192 samples. 

Let's now take past and present subsets and then compute the covariances using the formula above.

In [8]:
current_bin_combined = vcat(current_bin_187', current_bin_294');

state_past = current_bin_combined[:, 1:(T - tau)];
state_present = current_bin_combined[:, (1 + tau):T];

mean_past = mean(state_past, 2);
mean_present = mean(state_present, 2);

mean_past_array = repeat(mean_past, inner = [1, (T - tau)]);
mean_present_array = repeat(mean_present, inner = [1, (T - tau)]);

cov_present = ((state_present - mean_present_array) *
               (state_present - mean_present_array)') / (T - tau);

cov_cross = ((state_present - mean_present_array) *
             (state_past - mean_past_array)') / (T - tau);

println(cov_present)
println(cov_cross)




Float32[594.854 -1071.22; -1071.22 6124.42]
Float32[520.528 -1333.21; -700.807 5144.79]


# Calculating $\Phi$\* on a Single Trial

To calculate phi, we will make use of phi_gauss.m from the practical_phi toolbox (Oizumi). 

To use this, one method is to call Matlab from within Julia. This will require the MATLAB package; if this is not already installed, you will have to run `Pkg.add("MATLAB")` from within Julia before proceeding.

In [23]:
phi_gauss(cov_present, cov_cross)

LoadError: LoadError: Base.LinAlg.LAPACKException(1)
while loading In[23], in expression starting on line 1

In [ ]:
using MATLAB

phi, I, H = mxcall(:phi_gauss, 3, cov_present, cov_cross)

println(phi)
println(I)
println(H)

# Iterating Over All Trials


# Appendix


## cov_cond.jl

Ported from practical_phi toolbox's cov_cond.m (Oizumi).

In [9]:
function cov_cond(Cov_X, Cov_XY, Cov_Y)
    
    # Compute partial covariance of X given Y
    # Ported to Julia from practical_phi toolbox (Oizumi)
    # Here, we can ONLY make the assumption that Cov_Y = Cov_X. This is for 
    # convenience of porting at the moment, but may be changed later.
    
    # INPUTS:
    # Cov_X and Cov_Y are covariance matrices (default Cov_Y = Cov_X)
    # Cov_XY is a cross-covariance matrix of X and Y
    # OUTPUTS
    # Cov_X_Y: partial covariance of X given Y

    Cov_Y = Cov_X;

    Cov_X_Y = Cov_X - Cov_XY/Cov_Y*Cov_XY';
    
    return Cov_X_Y
    
end


cov_cond (generic function with 1 method)

## logdet.jl

Ported from practical_phi toolbox's logdet.m

In [10]:
function logdet(X,Const=0)

    # Compute log of determinant of X
    # Ported from logdet.m (Oizumi)


    X = float(X/10)^(float(Const));

    N = size(X,1);
    logdet_X = log(det(X)) + Const*N*log(10);
    
    return logdet_X
    
end

logdet (generic function with 2 methods)

## entropy_gauss.jl

Ported from practical_phi toolbox's entropy_gauss.m (Oizumi).

In [11]:
function entropy_gauss(Cov_X, Const=0)
    
    # Computes entropy under gaussian assumption
    # Ported to Julia from entropy_gauss.m (Oizumi)
    # INPUTS
    # Cov_X: covariance of data X
    # Const: Cov_X is divided by 10 ^ Const (default: 0)
    # OUTPUTS
    # H: entropy of X


    n = size(Cov_X, 1);
    H = 1/2*logdet(Cov_X, Const) + 1/2*n*log(2*pi*exp(1));
    
    return H
    
end

entropy_gauss (generic function with 2 methods)

## trace.jl

To replace Matlab's trace function

In [12]:
function trace_mat(matrix)
    
    return (sum(Diagonal(matrix)))
    
end


trace_mat (generic function with 1 method)

## phi_gauss.m

Ported from practical_phi's phi_gauss.m (Oizumi).

In [22]:
function phi_gauss(Cov_X, Cov_XY, optional...)
    
    # Computes practical measure of integrated information under assumption 
    # that probability distribution of data is Gaussian
    # Ported to Julia from phi_gauss.m (Oizumi)
    
    # Inputs:
    # X and Y represent the present and past state of the system respectively
    # Cov_X: Covariance matrix of X (present)
    # Cov_XY: Cross-covariance matrix of X (present) and Y (past)
    # Z: labels of elements according to which system is partitioned 
    # (default: atomic partition e.g. Z = [1 2] for 2-element system)
    # Cov_Y: covariance of Y (past state) (default: Cov_Y = Cov_X)
    # paras: parameters for gradient ascent
    # -- paras = [beta_init gamma iter_max threshold]
    # -- beta_init: initial value of beta
    # -- gamma: the speed of gradient ascent. If this value gets bigger, 
    # -- the convergence get's faster, but it may become unstable
    # -- iter_max: the maximum number of iteration
    # -- threshold: iteration stops once residual reaches this threshold value

    # OUTPUT
    # phi_star: practical measure of integrated information based on mismatched
    # decoding (Oizumi et al. 2016)
    # I: mutual information
    # H_cond: conditional entropy, H(X|Y)
    # phi_MI: Barrett and Seth's measure of phi based on MI (Barrett $ Seth 2011)
    # phi_H: Barrett and Seth's measure of phi based on conditional entropy 
    # (Barrett and Seth 2011)
    #beta: optimised beta value that maximises I*


    # REFERENCE: Oizumi M, Amari S-i, Yanagawa T, Fujii N, Tsuchiya N (2016) 
    # Measuring Integrated Information from the Decoding Perspective. 
    # PLoS Comput Biol 12(1): e1004654. doi:10.1371/journal.pcbi.1004654


    N = size(Cov_X,1); # number of elements
    if length(optional) < 1
        Z = collect(1: 1: N);
    else
        Z = optional[1]
    end
    if length(optional) < 2
        Cov_Y = Cov_X;
    else
        Cov_Y = optional[2]
    end
    if length(optional) < 3
        beta_init = 1;
        gamma = 10^-3.0;
        iter_max = 10^4.0;
        threshold = 10^-3.0;
    else
        paras = optional[3]
        beta_init = paras[1];
        gamma = paras[2];
        iter_max = paras[3];
        threshold = paras[4];
    end

    Cov_X_Y = cov_cond(Cov_X,Cov_XY,Cov_Y); # conditional covariance matrix
    Cov_Y_X = cov_cond(Cov_Y,Cov_XY',Cov_X); # conditional covariance matrix
    H_cond2 = entropy_gauss(Cov_Y_X,-1);

    H_cond = entropy_gauss(Cov_X_Y,-1); # conditional entropy in the whole system
    if isinf(H_cond) == 1
        println("Alert: Infinite Entropy\n")
    end
    if imag(H_cond) != 0
        println("Alert: Complex Entropy\n")
    end
    I = entropy_gauss(Cov_X,-1) - H_cond; # mutual information

    N_c = maximum(Z); # number of subsystems
    H_cond_p = zeros(N_c,1);
    H_cond_p2 = zeros(N_c,1);
    I_p = zeros(N_c,1);
    H_p = zeros(N_c,1);

    M_cell = Array{Any}(N_c);
    for i=1: N_c
        M_cell[i] = find(Z==i);
    end

    X_D = zeros(N,N); # Cov_D(X^t-tau)
    XX_D = zeros(N,N); # Cov_D(X^t,X^t-tau)
    C_D_cond = zeros(N,N);
    for i=1: N_c
        M = M_cell[i];
        Cov_X_p = Cov_X[M,M];
        Cov_Y_p = Cov_Y[M,M];
        Cov_XY_p = Cov_XY[M,M];
        Cov_X_Y_p = cov_cond(Cov_X_p,Cov_XY_p,Cov_Y_p);

        H_cond_p[i] = entropy_gauss(Cov_X_Y_p);
        H_p[i] = entropy_gauss(Cov_X_p);
        I_p[i] = H_p[i] - H_cond_p[i];

        X_D[M,M] = Cov_Y_p;
        XX_D[M,M] = Cov_XY_p;
        C_D_cond[M,M] = Cov_X_Y_p;

        Cov_Y_X_p = cov_cond(Cov_Y_p,Cov_XY_p',Cov_X_p);
        H_cond_p2[i] = entropy_gauss(Cov_Y_X_p);
    end
    H_cond_D = sum(H_cond_p);

    S = inv(C_D_cond);
    Cov_Y_inv = inv(Cov_Y);

    C_D_beta1_inv = X_D\XX_D'*S*XX_D/X_D;
    S_left = S'*XX_D/X_D;
    S_right = X_D\XX_D'*S;

    I_s_d_Const_part = trace_mat(inv(C_D_cond)*XX_D*inv(X_D)*Cov_XY);

    # find beta by gradient ascent
    beta = beta_init;
    for iter=1: iter_max
        C_D_beta_inv = beta*C_D_beta1_inv;
        Q_inv = Cov_Y_inv + C_D_beta_inv;
        Q = inv(Q_inv);

        Q_d = -Q*C_D_beta1_inv*Q; #derivative of Q
        R_d = -C_D_beta1_inv - beta*S_left*(2*Q + beta*Q_d)*S_right; # derivative of R

        I_s_d = 1/2*(-trace_mat(Q_inv*Q_d) + trace(Cov_X*R_d)) + I_s_d_Const_part; # derivative of I_s

        # println("iter=%d beta=%f I_s_d=%f\n",iter, beta,I_s_d);

        if abs(I_s_d) < threshold
            # fprintf("Iteration stopped at iter=%d. beta=%f\n",iter,beta);
            break;
        end

        beta = beta + gamma*I_s_d;

    end

    if iter == iter_max
        # println("Iteration did not converge. I_s_d=%f beta=%f\n",I_s_d,beta);
    end

    ## compute I* with optimized beta
    norm_t = -1/2*logdet(Q) + 1/2*logdet(Cov_Y);
    R = - beta*C_D_beta1_inv - beta^2*S_left*Q*S_right;

    trace_t = 1/2*trace_mat(Cov_X*R);
    I_s = norm_t + trace_t + beta*I_s_d_Const_part;
    println("beta=%f I_s=%f\n",beta,I_s);


    K = (N_c-1)*min(H_p); # normalization proposed in Balduzzi & Tononi, 2008, PLoS Comp Biol

    ## 

    phi_H_raw = sum(H_cond_p2) - H_cond2; # conditional entropy-based integrated information
    phi_H_norm = phi_H[1]/K;
    phi_H = (phi_H_raw, phi_H_norm)

    phi_MI_raw = I - sum(I_p); # mutual information-based integrated information
    phi_MI_norm = phi_MI[1]/K;
    phi_MI = (phi_MI_raw, phi_MI_norm)

    phi_star_raw = I - I_s; # our proposed measure of integrated information
    phi_star_norm = phi_star[1]/K;
    phi_star = (phi_star_raw, phi_star_norm)
                    
    return phi_star, I, H_cond, phi_MI, phi_H, beta

end

phi_gauss (generic function with 1 method)

Any, Any, Any...) in module Main at In[20]:38 overwritten at In[22]:38.
